In [ ]:
%load_ext lab_black

import h5py
import os
import numpy as np
import pandas as pd
from typing import Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm


import plotly.graph_objects as go
import plotly.colors as pc

# plt.style.use('ggplot')
import plotly.io as pio

pio.templates.default = "plotly_white"
pio.renderers.default = "jupyterlab"
import matplotlib.pyplot as plt

import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    tr_functions,
)


colors = pc.qualitative.D3
angstrom = "\u212B"
degree = "\u00B0"

In [ ]:
default_layout = dict(
    template="plotly_white",
    xaxis=dict(
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
    ),
    yaxis=dict(
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
    ),
)

In [ ]:
fp = r"E:\atully\QCM data"

# QCM Data: June 2023

## QCM 1: Stage 1

In [ ]:
title = "QCM #1"
data_path = os.path.join(fp, "22_06_2023_QCM1.csv")
df = pd.read_csv(data_path, header=1, sep=";")  # 2nd line of csv file is headers
df = df.drop(26990)
df

In [ ]:
# Get data from df

x_data = df.iloc[:, 0][::50]  # get every 50th datapoint
y_data = df.iloc[:, 2][::50]

In [ ]:
import datetime

x_datanew = x_data.apply(
    lambda datestring: pd.Timestamp(
        datetime.datetime.strptime(datestring[:-4], "%H:%M:%S")
    )
)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_datanew, y=y_data, mode="lines", name="data"))

fig.update_layout(
    xaxis_tickformat="%H:%M",
    xaxis_title="Time (min)",
    yaxis_title="Thickness (nm)",
    title=title,
    **default_layout
)

In [ ]:
xtry = x_datanew.apply(lambda x: float(x.value))
xtry -= min(xtry)  # get rid of offset from unix time
xtry /= 1e9  # ns to s
xtry /= 60  # s to min
xtry

In [ ]:
fit1 = fitting_functions.fit_linear_data(
    x=np.array(xtry), data=np.array(y_data), num=1, aes=1, bes=1, offset_type=None
)
fig.add_trace(
    go.Scatter(
        x=x_datanew,
        y=fit1.eval(x=xtry),
        mode="lines",
        name="linear fit",
        line=dict(color="red", dash="dash"),
    )
)
fig.show()
print(
    f"slope: {np.round(fit1.values['i0_slope'],3)} nm/min, intercept: {np.round(fit1.values['i0_intercept'], 3)} nm"
)
print(f"slope * 3 hours = {np.round(fit1.values['i0_slope']*3*60,3)} nm")

## QCM 2: Stage 1

In [ ]:
title = "QCM #2"
data_path = os.path.join(fp, "22_06_2023_QCM2.csv")
df = pd.read_csv(data_path, header=1, sep=";")  # 2nd line of csv file is headers
df = df.drop(26990)
df

In [ ]:
# Get data from df

x_data = df.iloc[:, 0][::50]  # get every 50th datapoint
y_data = df.iloc[:, 2][::50]

In [ ]:
import datetime

x_datanew = x_data.apply(
    lambda datestring: pd.Timestamp(
        datetime.datetime.strptime(datestring[:-4], "%H:%M:%S")
    )
)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_datanew, y=y_data, mode="lines", name="data"))

fig.update_layout(
    xaxis_tickformat="%H:%M",
    xaxis_title="Time (min)",
    yaxis_title="Thickness (nm)",
    title=title,
    **default_layout
)

In [ ]:
xtry = x_datanew.apply(lambda x: float(x.value))
xtry -= min(xtry)  # get rid of offset from unix time
xtry /= 1e9  # ns to s
xtry /= 60  # s to min
xtry

In [ ]:
fit2 = fitting_functions.fit_linear_data(
    x=np.array(xtry), data=np.array(y_data), num=1, aes=1, bes=1, offset_type=None
)
fig.add_trace(
    go.Scatter(
        x=x_datanew,
        y=fit2.eval(x=xtry),
        mode="lines",
        name="linear fit",
        line=dict(color="red", dash="dash"),
    )
)
fig.show()
print(
    f"slope: {np.round(fit2.values['i0_slope'],3)} nm/min, intercept: {np.round(fit2.values['i0_intercept'], 3)} nm"
)
print(f"slope * 3 hours = {np.round(fit2.values['i0_slope']*3*60,3)} nm")